In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

Load dataset

In [5]:
data = pd.read_csv('career_dataset.csv')
print(data.head())
print(data.info())

   CandidateID           Name  Age   Education  \
0            1       John Doe   28  Bachelor's   
1            2     Jane Smith   32    Master's   
2            3    Bob Johnson   24  Bachelor's   
3            4    Emily Davis   26  Bachelor's   
4            5  Michael Brown   30    Master's   

                                      Skills                Interests  \
0      Python;Data Analysis;Machine Learning  Technology;Data Science   
1         Java;System Design;Cloud Computing  Software Development;AI   
2  Graphic Design;UI/UX;Adobe Creative Suite       Arts;Digital Media   
3            Python;Deep Learning;Statistics            Healthcare;AI   
4     Project Management;Communication;Agile      Business;Management   

  Recommended_Career  Recommendation_Score  
0     Data Scientist                  0.95  
1  Software Engineer                  0.90  
2        UX Designer                  0.88  
3      AI Researcher                  0.93  
4    Project Manager               

Preprocessing

In [6]:
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

In [7]:
data = data.drop(['CandidateID', 'Name'], axis=1)

In [8]:
le_edu = LabelEncoder()
data['Education'] = le_edu.fit_transform(data['Education'])

In [9]:
mlb_skills = MultiLabelBinarizer()
data_skills = mlb_skills.fit_transform(data['Skills'].str.split(';'))
skills_df = pd.DataFrame(data_skills, columns=mlb_skills.classes_)
data = pd.concat([data, skills_df], axis=1)
data = data.drop('Skills', axis=1)
mlb_interests = MultiLabelBinarizer()
data_interests = mlb_interests.fit_transform(data['Interests'].str.split(';'))
interests_df = pd.DataFrame(data_interests, columns=mlb_interests.classes_)
data = pd.concat([data, interests_df], axis=1)
data = data.drop('Interests', axis=1)

Features and Target

In [10]:
X = data.drop(['Recommended_Career', 'Recommendation_Score'], axis=1)
le_career = LabelEncoder()
y = le_career.fit_transform(data['Recommended_Career'])


Train-Test Split

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Model Train

In [12]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

Evaluate the Model

In [14]:
import numpy as np
unique_test_classes = np.unique(y_test)
print(unique_test_classes)
print([le_career.classes_[i] for i in unique_test_classes])


[ 2  3  4  5  6  7  8 10 12 13 15 16 17 18 19 20 21 23 24 25 26 27 28 30
 31]
['Automation Engineer', 'Backend Developer', 'Biostatistician', 'Business Analyst', 'Cloud Engineer', 'Content Strategist', 'Cybersecurity Analyst', 'Data Analyst', 'Data Scientist', 'Deep Learning Engineer', 'Digital Marketer', 'Embedded Systems Engineer', 'Financial Analyst', 'Front-end Developer', 'Full Stack Developer', 'Graphic Designer', 'Machine Learning Engineer', 'Mobile Developer', 'NLP Engineer', 'Project Manager', 'Research Analyst', 'Research Scientist', 'Software Developer', 'UX Designer', 'UX Researcher']


In [15]:
from sklearn.metrics import classification_report
labels = unique_test_classes
target_names = [le_career.classes_[i] for i in labels]
print(classification_report(y_test, y_pred, labels=labels, target_names=target_names))


                           precision    recall  f1-score   support

      Automation Engineer       0.50      1.00      0.67         1
        Backend Developer       0.50      1.00      0.67         1
          Biostatistician       0.00      0.00      0.00         2
         Business Analyst       1.00      0.50      0.67         2
           Cloud Engineer       0.00      0.00      0.00         1
       Content Strategist       0.00      0.00      0.00         1
    Cybersecurity Analyst       1.00      0.33      0.50         3
             Data Analyst       0.00      0.00      0.00         4
           Data Scientist       1.00      0.50      0.67         2
   Deep Learning Engineer       0.00      0.00      0.00         1
         Digital Marketer       1.00      0.60      0.75         5
Embedded Systems Engineer       0.33      1.00      0.50         1
        Financial Analyst       0.67      1.00      0.80         2
      Front-end Developer       1.00      1.00      1.00     

c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Make Predictions

In [17]:
new_candidate = {}
new_candidate['Age'] = 25
new_candidate['Education'] = le_edu.transform(['Bachelor\'s'])[0]

# For each skill column, set 1 if the candidate has it, else 0
skills = ['Python', 'Java', 'Graphic Design', ...]  # All skill columns from X
candidate_skills = ['Python', 'Graphic Design']     # Skills for this candidate
for skill in skills:
    new_candidate[skill] = 1 if skill in candidate_skills else 0

# For each interest column, set 1 if present, else 0
interests = ['AI', 'Data Science', ...]             # All interest columns from X
candidate_interests = ['AI']                        # Interests for this candidate
for interest in interests:
    new_candidate[interest] = 1 if interest in candidate_interests else 0


In [19]:
# Get all feature columns from your training set
feature_columns = X.columns.tolist()

# Candidate's actual info
candidate_info = {
    'Age': 25,
    'Education': le_edu.transform(['Bachelor\'s'])[0],
    # List all skills/interests the candidate has
    'skills': ['Python', 'Data Analysis'],
    'interests': ['Technology', 'Data Science']
}

# Build the full feature dictionary
new_candidate = {}
for col in feature_columns:
    if col == 'Age':
        new_candidate[col] = candidate_info['Age']
    elif col == 'Education':
        new_candidate[col] = candidate_info['Education']
    elif col in candidate_info['skills']:
        new_candidate[col] = 1
    elif col in candidate_info['interests']:
        new_candidate[col] = 1
    else:
        # For all other skill/interest columns not present, set to 0
        new_candidate[col] = 0

# Create DataFrame and ensure column order matches
new_df = pd.DataFrame([new_candidate])
new_df = new_df[X.columns]  # This should now work!


In [20]:
print("Expected columns:", X.columns.tolist())
print("Candidate keys:", new_candidate.keys())


Expected columns: ['Age', 'Education', '.NET', 'AI', 'Adobe Creative Suite', 'Adobe Illustrator', 'Adobe Photoshop', 'Adobe XD', 'Agile', 'Algorithms', 'Analytics', 'Android', 'Automation', 'Big Data', 'Business Analysis', 'C#', 'C++', 'CRM', 'CSS', 'Cloud Computing', 'Communication', 'Content Creation', 'Content Strategy', 'Content Writing', 'Copywriting', 'Creativity', 'Cybersecurity', 'Data Analysis', 'Data Mining', 'Data Science', 'Data Structures', 'Data Visualization', 'Data Warehousing', 'Deep Learning', 'DevOps', 'Digital Illustration', 'Digital Marketing', 'ETL', 'Econometrics', 'Embedded Systems', 'Excel', 'Financial Analysis', 'Graphic Design', 'HTML', 'Illustration', 'Interaction Design', 'IoT', 'Java', 'JavaScript', 'Linux', 'Machine Learning', 'Marketing Strategy', 'Microservices', 'NLP', 'Natural Language Processing', 'Negotiation', 'Network Security', 'Node.js', 'Project Management', 'Prototyping', 'Python', 'R', 'React', 'Research', 'Risk Analysis', 'SEO', 'SQL', 'Soci

In [21]:
pred = model.predict(new_df)
print('Recommended Career:', le_career.inverse_transform(pred)[0])


Recommended Career: Data Analyst


Accuracy

In [22]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.475


Precision, Recall, and F1 Score

In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)


Precision: 0.5333333333333333
Recall: 0.475
F1 Score: 0.45041666666666663


c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           2       0.50      1.00      0.67         1
           3       0.50      1.00      0.67         1
           4       0.00      0.00      0.00         2
           5       1.00      0.50      0.67         2
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       1.00      0.33      0.50         3
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         0
          12       1.00      0.50      0.67         2
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         0
          15       1.00      0.60      0.75         5
          16       0.33      1.00      0.50         1
          17       0.67      1.00      0.80         2
          18       1.00      1.00      1.00         1
          19       0.33      1.00      0.50         1
          20       1.00    

c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

Confusion Matrix

In [26]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)


[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 3 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Import Required Libraries

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier


In [5]:
pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Load Data & Preprocess

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

# -----------------------------
# Example: Load your dataset
# -----------------------------
# df = pd.read_csv("your_dataset.csv")

# Assuming your dataset is already in DataFrame `df`
# Target column = "Recommended_Career"
X = df.drop("Recommended_Career", axis=1)
y = df["Recommended_Career"]

# Encode target if categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -----------------------------
# Check class distribution
# -----------------------------
print("Class distribution before SMOTE:", pd.Series(y_train).value_counts())

# -----------------------------
# Apply SMOTE safely
# -----------------------------
# Use k_neighbors=2 to avoid "n_neighbors" error
smote = SMOTE(random_state=42, k_neighbors=2)
X_train, y_train = smote.fit_resample(X_train, y_train)

print("Class distribution after SMOTE:", pd.Series(y_train).value_counts())

# -----------------------------
# Scale features
# -----------------------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# -----------------------------
# Train model
# -----------------------------
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# -----------------------------
# Evaluation
# -----------------------------
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Class distribution before SMOTE: 10    17
15    10
20    10
18     8
25     7
30     7
21     7
17     7
7      7
5      6
28     6
12     6
27     5
16     4
24     4
4      3
14     3
29     3
26     3
1      3
3      3
31     3
19     3
6      3
11     3
8      3
2      3
23     3
0      3
22     3
13     2
9      2
Name: count, dtype: int64


ValueError: could not convert string to float: 'Adalyn Jenkins'

Train Model

In [10]:
# Define model
rf = RandomForestClassifier(random_state=42)

# Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_


c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


4. Evaluate Model

In [11]:
# Predictions
y_pred = best_model.predict(X_test)

# Metrics
print("Best Parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy: 0.35

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.50      1.00      0.67         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.25      1.00      0.40         1
           9       0.00      0.00      0.00         1
          10       0.50      0.50      0.50         4
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00   

c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

5. Save Model

In [14]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler
scaler = StandardScaler()

# Fit + transform training data, transform test data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [15]:
import joblib

joblib.dump(best_model, "career_model.pkl")
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

Check Accuracy

In [16]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predictions
y_pred = best_model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

# Detailed metrics
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Model Accuracy: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       1.0
           2       0.00      0.00      0.00       1.0
           3       0.00      0.00      0.00       1.0
           4       0.00      0.00      0.00       1.0
           5       0.00      0.00      0.00       1.0
           6       0.00      0.00      0.00       1.0
           7       0.00      0.00      0.00       1.0
           8       0.00      0.00      0.00       1.0
           9       0.00      0.00      0.00       1.0
          10       0.00      0.00      0.00       4.0
          11       0.00      0.00      0.00       1.0
          12       0.00      0.00      0.00       1.0
          13       0.00      0.00      0.00       1.0
          14       0.00      0.00      0.00       1.0
          15       0.00      0.00      0.00       2.0
          16       0.00      0.00   

c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packa

In [17]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['Education', 'Skills', 'Interests']
encoder = OneHotEncoder(handle_unknown='ignore')

X_encoded = encoder.fit_transform(df[categorical_cols])
X_encoded = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names_out(categorical_cols))

# Combine with numerical columns
X_final = pd.concat([df.drop(categorical_cols + ['CandidateID','Name','Recommended_Career','Recommendation_Score'], axis=1).reset_index(drop=True),
                     X_encoded.reset_index(drop=True)], axis=1)

y = df['Recommended_Career']


In [20]:
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [22:12:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.3

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.50      1.00      0.67         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.25      1.00      0.40         1
           9       0.00      0.00      0.00         1
          10       0.50      0.25      0.33         4
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          15       0.50      0.50      0.50         2
          16       0.00      0.00      0.0

c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [19]:
pip install xgboost


   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   -- ------------------------------------- 3.4/56.8 MB 20.2 MB/s eta 0:00:03
   ----- ---------------------------------- 7.6/56.8 MB 19.6 MB/s eta 0:00:03
   ------- -------------------------------- 11.3/56.8 MB 18.6 MB/s eta 0:00:03
   ---------- ----------------------------- 14.7/56.8 MB 18.5 MB/s eta 0:00:03
   ------------ --------------------------- 17.3/56.8 MB 17.3 MB/s eta 0:00:03
   ------------- -------------------------- 18.6/56.8 MB 15.5 MB/s eta 0:00:03
   -------------- ------------------------- 19.9/56.8 MB 14.1 MB/s eta 0:00:03
   --------------- ------------------------ 21.5/56.8 MB 13.3 MB/s eta 0:00:03
   ---------------- ----------------------- 23.3/56.8 MB 12.8 MB/s eta 0:00:03
   ----------------- ---------------------- 24.9/56.8 MB 12.2 MB/s eta 0:00:03
   ------------------- -------------------- 27.3/56.8 MB 12.1 MB/s eta 0:00:03
   -------------------- ------------------- 29.6/56.8 MB 12.0 M


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Cross-Validation

In [21]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(best_model, X, y, cv=5, scoring='accuracy')
print("Cross-validation accuracy:", scores)
print("Mean Accuracy:", scores.mean())


c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation accuracy: [0.725 0.75  0.375 0.025 0.   ]
Mean Accuracy: 0.375


Accuracy

In [23]:
import joblib
from sklearn.metrics import accuracy_score

# Load saved model and scaler
model = joblib.load("career_model.pkl")   # <-- this is the actual trained model
scaler = joblib.load("scaler.pkl")       # <-- this is just the scaler

# Transform X_test with the saved scaler
X_test_scaled = scaler.transform(X_test)

# Predict
y_pred = model.predict(X_test_scaled)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Exact Model Accuracy: {accuracy:.4f}")


Exact Model Accuracy: 0.0250


c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


1. Use One-Hot Encoding

In [24]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['Education', 'Skills', 'Interests']
encoder = OneHotEncoder(handle_unknown='ignore')

X_encoded = encoder.fit_transform(df[categorical_cols])
X_encoded = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names_out(categorical_cols))

# Combine with other numeric features
X_final = pd.concat([df.drop(categorical_cols + ['CandidateID','Name','Recommended_Career','Recommendation_Score'], axis=1).reset_index(drop=True),
                     X_encoded.reset_index(drop=True)], axis=1)

y = df['Recommended_Career']


2. Use Stratified Train-Test Split

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.3, random_state=42, stratify=y
)

3. Train with XGBoost (better for structured categorical data)

In [26]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)
print("Exact Model Accuracy:", accuracy_score(y_test, y_pred))


c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [22:15:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Exact Model Accuracy: 0.1


In [27]:
xgb = XGBClassifier(eval_metric='mlogloss', random_state=42)
xgb.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [28]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgb, X_final, y, cv=5, scoring='accuracy')
print("Cross-validation scores:", scores)
print("Mean Accuracy:", scores.mean())


c:\Users\vansh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation scores: [0.35  0.25  0.25  0.05  0.025]
Mean Accuracy: 0.185
